In [ ]:
from rply import LexerGenerator 

class Lexer:
    def __init__(self):
        self.lexer = LexerGenerator() 

    def _add_tokens(self):
        self.lexer.add('PRINT', r'showing')
        self.lexer.add('OPEN_PAREN', r'\(')
        self.lexer.add('CLOSE_PAREN', r'\)')
        self.lexer.add('SEMI_COLON', r'\;')
        self.lexer.add('SUM', r'jog')
        self.lexer.add('SUB', r'biyog')
        self.lexer.add('MUL', r'gun')
        self.lexer.add('DIV', r'vag')
        self.lexer.add('NUMBER', r'\d+')
        self.lexer.add('LITERAL', r'"([^"\\]|\\.)*"')
        self.lexer.ignore('\s+') 

    def get_lexer(self):
        self._add_tokens()
        return self.lexer.build()

class Number:
    def __init__(self, value):
        self.value = value  
    def eval(self):
        return int(self.value)  

class BinaryOp:
    def __init__(self, left, right):
        self.left = left  
        self.right = right  

class Sum(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()  

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()  

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()  

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval() 

class Print:
    def __init__(self, value):
        self.value = value  

    def eval(self):
        if isinstance(self.value, str): 
            print(self.value.strip('"')) 
        else:
            print(self.value.eval()) 

from rply import ParserGenerator

class Parser:
    def __init__(self):
        self.pg = ParserGenerator(
            ['NUMBER', 'LITERAL', 'PRINT',  'OPEN_PAREN', 'CLOSE_PAREN',
            'SEMI_COLON', 'SUM', 'SUB', 'MUL', 'DIV']
        )

    def parse(self):
        @self.pg.production('program : statement SEMI_COLON')
        def program(p):
            return p[0]

        @self.pg.production('statement : PRINT OPEN_PAREN expression CLOSE_PAREN')
        def print_statement(p):
            return Print(p[2])

        @self.pg.production('factor : OPEN_PAREN expression CLOSE_PAREN')
        def factor_parenthesis(p):
            return p[1]

        @self.pg.production('expression : expression SUM term')
        @self.pg.production('expression : expression SUB term')
        def expression(p):
            left = p[0]
            right = p[2]
            operator = p[1]
            if operator.gettokentype() == 'SUM':
                return Sum(left, right)
            elif operator.gettokentype() == 'SUB':
                return Sub(left, right)

        @self.pg.production('expression : term')
        def expression_term(p):
            return p[0]

        @self.pg.production('term : term MUL factor')
        @self.pg.production('term : term DIV factor')
        def term(p):
            left = p[0]
            right = p[2]
            operator = p[1] 
            if operator.gettokentype() == 'MUL':
                return Mul(left, right) 
            elif operator.gettokentype() == 'DIV':
                return Div(left, right)

        @self.pg.production('term : factor')
        def term_factor(p):
            return p[0]

        @self.pg.production('factor : NUMBER')
        def factor_number(p):
            return Number(p[0].value) 

        @self.pg.production('factor : LITERAL')
        def factor_literal(p):
            return p[0].value 

        @self.pg.error
        def error_handle(token):
            raise ValueError(token)

    def get_parser(self):
        return self.pg.build() 


def token(input):
    lexer = Lexer().get_lexer()
    tokens = lexer.lex(input) 
    for token in tokens:
        print(f"Lexeme '{token.getstr().lower()}'\t matches token name = '{token.gettokentype().replace('_', ' ')}'")

def run(input):
    lexer = Lexer().get_lexer()
    tokens = lexer.lex(input) 
    token(input) 
    pg = Parser() 
    pg.parse() 
    parser = pg.get_parser() 
    print('\nAfter Compiling the code: ')
    parser.parse(tokens).eval() 
    

text_input = 'showing("this is Rakib Compilar");' 
run(text_input)


In [ ]:

text_input = 'showing(4 jog 5);'
run(text_input)  


In [ ]:

text_input = 'showing(4 jog (3 biyog 2) gun (5 jog 5));' 
run(text_input) 
